In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from keras.metrics import AUC
from keras.models import Sequential
from keras.layers import Dense

In [2]:
# Function that finds the FDR at 3%
def findFDR(y_label, y_pred):
    data = pd.DataFrame({'Actual':y_label, 'PredictedProb':y_pred})
    topRows = int(round(len(data)*0.03))
    temp0 = data.sort_values('PredictedProb',ascending=False)
    temp1 = temp0.head(topRows)
    numbads = sum(temp0.loc[:, 'Actual'])
    needed1 = temp1.loc[:,'Actual']
    FDR1 = sum(needed1)/numbads * 100
    print('FDR at 3% rate: ', round(FDR1, 2))

In [3]:
data = pd.read_csv('application data.csv', parse_dates = ['date'])

In [4]:
# Sets OOT data
oot = data.loc[data['date'] > '2016-10-31']
traintest = data.loc[data['date'] <= '2016-10-31']
X = traintest.drop('date', axis = 1)
X = X.drop('fraud_label', axis = 1)
y = traintest['fraud_label']
ootX = oot.drop('date', axis = 1)
ootX = ootX.drop('fraud_label', axis = 1)
ootY = oot['fraud_label']

In [5]:
# Normalizes (z-scale) the paratio and rm values
mms = MinMaxScaler()

X = mms.fit_transform(X)
ootX = mms.fit_transform(ootX)

# Sampling Data

In [6]:
# Splits data into 75/25 train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [7]:
# Creates different undersampling ratios
rus = RandomUnderSampler(sampling_strategy = 1)
X_train_under50, y_train_under50 = rus.fit_sample(X_train, y_train)

rus = RandomUnderSampler(sampling_strategy = (2/3))
X_train_under40, y_train_under40 = rus.fit_sample(X_train, y_train)

rus = RandomUnderSampler(sampling_strategy = (3/7))
X_train_under30, y_train_under30 = rus.fit_sample(X_train, y_train)

In [8]:
# Creates different oversampling ratios
oversample = SMOTE(sampling_strategy = 1)
X_train_over50, y_train_over50 = oversample.fit_resample(X_train, y_train)

oversample = SMOTE(sampling_strategy = (2/3))
X_train_over40, y_train_over40 = oversample.fit_resample(X_train, y_train)

oversample = SMOTE(sampling_strategy = (3/7))
X_train_over30, y_train_over30 = oversample.fit_resample(X_train, y_train)

# Logistic Regression

In [14]:
model = LogisticRegression().fit(X_train, y_train)

/Users/kpainton/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [15]:
y_pred = model.predict_proba(X_train)
findFDR(y_train, y_pred[:,1])

FDR at 3% rate:  52.61


In [16]:
y_pred = model.predict_proba(X_test)
findFDR(y_test, y_pred[:,1])

FDR at 3% rate:  52.23


In [17]:
y_pred = model.predict_proba(ootX)
findFDR(ootY, y_pred[:,1])

FDR at 3% rate:  50.29


# Neural Networks

In [26]:
# Defines a simple Neural Network model function 
def runNN(X, y, nodes, optimizer, epochs):
    # first neural network with keras make predictions

    # define the keras model
    model = Sequential()
    model.add(Dense(nodes, input_dim=30, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # compile the keras model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[AUC()])

    # fit the keras model on the dataset
    model.fit(X, y, epochs=epochs, batch_size=10)

    y_pred = model.predict(X_train)
    print('Train')
    findFDR(y_train, y_pred.reshape(-1))

    y_pred = model.predict(X_test)
    print('Test')
    findFDR(y_test, y_pred.reshape(-1))

    y_pred = model.predict(ootX)
    print('OOT')
    findFDR(ootY, y_pred.reshape(-1))

In [ ]:
data.columns

In [39]:
runNN(X_train_over30, y_train_over30, 10, 'sgd', 20)

Epoch 1/20
88019/88019 [==============================] - 45s 506us/step - loss: 0.4263 - auc_5: 0.7673
Epoch 2/20
88019/88019 [==============================] - 44s 498us/step - loss: 0.4060 - auc_5: 0.7711
Epoch 3/20
88019/88019 [==============================] - 44s 500us/step - loss: 0.4029 - auc_5: 0.7720
Epoch 4/20
88019/88019 [==============================] - 43s 488us/step - loss: 0.4014 - auc_5: 0.7714
Epoch 5/20
88019/88019 [==============================] - 43s 489us/step - loss: 0.4002 - auc_5: 0.7724
Epoch 6/20
88019/88019 [==============================] - 43s 488us/step - loss: 0.3997 - auc_5: 0.7722
Epoch 7/20
88019/88019 [==============================] - 43s 489us/step - loss: 0.3994 - auc_5: 0.7725
Epoch 8/20
88019/88019 [==============================] - 43s 488us/step - loss: 0.3992 - auc_5: 0.7721
Epoch 9/20
88019/88019 [==============================] - 43s 489us/step - loss: 0.3989 - auc_5: 0.7720
Epoch 10/20
88019/88019 [==============================] - 43s 4

In [40]:
runNN(X_train_over30, y_train_over30, 30, 'sgd', 20)

Epoch 1/20
88019/88019 [==============================] - 43s 490us/step - loss: 0.4237 - auc_6: 0.7680
Epoch 2/20
88019/88019 [==============================] - 43s 490us/step - loss: 0.4067 - auc_6: 0.7723
Epoch 3/20
88019/88019 [==============================] - 43s 490us/step - loss: 0.4036 - auc_6: 0.7727
Epoch 4/20
88019/88019 [==============================] - 43s 490us/step - loss: 0.4008 - auc_6: 0.7720
Epoch 5/20
88019/88019 [==============================] - 43s 490us/step - loss: 0.3997 - auc_6: 0.7718
Epoch 6/20
88019/88019 [==============================] - 43s 490us/step - loss: 0.3992 - auc_6: 0.7720
Epoch 7/20
88019/88019 [==============================] - 44s 503us/step - loss: 0.3988 - auc_6: 0.7716
Epoch 8/20
88019/88019 [==============================] - 43s 489us/step - loss: 0.3984 - auc_6: 0.7728
Epoch 9/20
88019/88019 [==============================] - 43s 485us/step - loss: 0.3982 - auc_6: 0.7725
Epoch 10/20
88019/88019 [==============================] - 43s 4

In [41]:
runNN(X_train_over30, y_train_over30, 30, 'sgd', 30)

Epoch 1/30
88019/88019 [==============================] - 43s 484us/step - loss: 0.4239 - auc_7: 0.7688
Epoch 2/30
88019/88019 [==============================] - 43s 485us/step - loss: 0.4078 - auc_7: 0.7718
Epoch 3/30
88019/88019 [==============================] - 43s 485us/step - loss: 0.4035 - auc_7: 0.7725
Epoch 4/30
88019/88019 [==============================] - 43s 485us/step - loss: 0.4014 - auc_7: 0.7725
Epoch 5/30
88019/88019 [==============================] - 43s 485us/step - loss: 0.4004 - auc_7: 0.7722
Epoch 6/30
88019/88019 [==============================] - 43s 484us/step - loss: 0.3998 - auc_7: 0.7726
Epoch 7/30
88019/88019 [==============================] - 44s 496us/step - loss: 0.3992 - auc_7: 0.7725
Epoch 8/30
88019/88019 [==============================] - 43s 491us/step - loss: 0.3989 - auc_7: 0.7726
Epoch 9/30
88019/88019 [==============================] - 43s 491us/step - loss: 0.3985 - auc_7: 0.7729
Epoch 10/30
88019/88019 [==============================] - 43s 4

In [42]:
runNN(X_train_over30, y_train_over30, 50, 'adam', 30)

Epoch 1/30
88019/88019 [==============================] - 47s 535us/step - loss: 0.3964 - auc_8: 0.7727
Epoch 2/30
88019/88019 [==============================] - 47s 538us/step - loss: 0.3901 - auc_8: 0.7742
Epoch 3/30
88019/88019 [==============================] - 47s 535us/step - loss: 0.3893 - auc_8: 0.7740
Epoch 4/30
88019/88019 [==============================] - 47s 535us/step - loss: 0.3889 - auc_8: 0.7741
Epoch 5/30
88019/88019 [==============================] - 47s 536us/step - loss: 0.3888 - auc_8: 0.7747
Epoch 6/30
88019/88019 [==============================] - 47s 538us/step - loss: 0.3886 - auc_8: 0.7750
Epoch 7/30
88019/88019 [==============================] - 50s 563us/step - loss: 0.3885 - auc_8: 0.7752
Epoch 8/30
88019/88019 [==============================] - 47s 536us/step - loss: 0.3883 - auc_8: 0.7756
Epoch 9/30
88019/88019 [==============================] - 47s 535us/step - loss: 0.3883 - auc_8: 0.7755
Epoch 10/30
88019/88019 [==============================] - 47s 5

In [43]:
runNN(X_train_over30, y_train_over30, 30, 'adam', 40)

Epoch 1/40
88019/88019 [==============================] - 48s 548us/step - loss: 0.3995 - auc_9: 0.7724
Epoch 2/40
88019/88019 [==============================] - 48s 549us/step - loss: 0.3911 - auc_9: 0.7740
Epoch 3/40
88019/88019 [==============================] - 48s 549us/step - loss: 0.3893 - auc_9: 0.7747
Epoch 4/40
88019/88019 [==============================] - 48s 549us/step - loss: 0.3888 - auc_9: 0.7745
Epoch 5/40
88019/88019 [==============================] - 48s 549us/step - loss: 0.3886 - auc_9: 0.7752
Epoch 6/40
88019/88019 [==============================] - 48s 548us/step - loss: 0.3884 - auc_9: 0.7759
Epoch 7/40
88019/88019 [==============================] - 48s 548us/step - loss: 0.3883 - auc_9: 0.7755
Epoch 8/40
88019/88019 [==============================] - 48s 549us/step - loss: 0.3882 - auc_9: 0.7753
Epoch 9/40
88019/88019 [==============================] - 48s 549us/step - loss: 0.3882 - auc_9: 0.7751
Epoch 10/40
88019/88019 [==============================] - 48s 5

In [44]:
runNN(X_train_over30, y_train_over30, 50, 'adam', 40)

Epoch 1/40
88019/88019 [==============================] - 47s 539us/step - loss: 0.3973 - auc_10: 0.7725
Epoch 2/40
88019/88019 [==============================] - 47s 537us/step - loss: 0.3904 - auc_10: 0.7743
Epoch 3/40
88019/88019 [==============================] - 47s 538us/step - loss: 0.3892 - auc_10: 0.7739
Epoch 4/40
88019/88019 [==============================] - 47s 538us/step - loss: 0.3888 - auc_10: 0.7748
Epoch 5/40
88019/88019 [==============================] - 47s 539us/step - loss: 0.3886 - auc_10: 0.7747
Epoch 6/40
88019/88019 [==============================] - 47s 538us/step - loss: 0.3885 - auc_10: 0.7751
Epoch 7/40
88019/88019 [==============================] - 48s 540us/step - loss: 0.3884 - auc_10: 0.7750
Epoch 8/40
88019/88019 [==============================] - 48s 544us/step - loss: 0.3883 - auc_10: 0.7749
Epoch 9/40
88019/88019 [==============================] - 47s 539us/step - loss: 0.3882 - auc_10: 0.7756
Epoch 10/40
88019/88019 [==============================

In [52]:
# Last Run
runNN(X_train_over30, y_train_over30, 50, 'adam', 20)

Epoch 1/20
88019/88019 [==============================] - 48s 545us/step - loss: 0.3974 - auc_11: 0.7724
Epoch 2/20
88019/88019 [==============================] - 48s 544us/step - loss: 0.3902 - auc_11: 0.7737
Epoch 3/20
88019/88019 [==============================] - 48s 545us/step - loss: 0.3891 - auc_11: 0.7740
Epoch 4/20
88019/88019 [==============================] - 48s 544us/step - loss: 0.3889 - auc_11: 0.7744
Epoch 5/20
88019/88019 [==============================] - 48s 543us/step - loss: 0.3886 - auc_11: 0.7752
Epoch 6/20
88019/88019 [==============================] - 48s 544us/step - loss: 0.3886 - auc_11: 0.7754
Epoch 7/20
88019/88019 [==============================] - 48s 544us/step - loss: 0.3885 - auc_11: 0.7753
Epoch 8/20
88019/88019 [==============================] - 48s 544us/step - loss: 0.3884 - auc_11: 0.7752
Epoch 9/20
88019/88019 [==============================] - 48s 544us/step - loss: 0.3883 - auc_11: 0.7754
Epoch 10/20
88019/88019 [==============================

# Random Forest

In [9]:
# Defines a simple Random Forest 
def RunForest(trees,depth, min_sample, criterion, X, y): 
    model = RandomForestClassifier(n_estimators = trees,
                                   min_samples_split = min_sample, 
                                   max_depth = depth,
                                   class_weight = {1:.9, 0:.1}
                                  ).fit(
        X, y)
    y_pred = model.predict_proba(X_train)
    print("Train")
    findFDR(y_train, y_pred[:,1])
    y_pred = model.predict_proba(X_test)
    print('Test')
    findFDR(y_test, y_pred[:,1])
    y_pred = model.predict_proba(ootX)
    print('OOT')
    findFDR(ootY, y_pred[:,1])

In [10]:
RunForest(100, 20, 300, 'gini', X_train, y_train)

Train
FDR at 3% rate:  55.54
Test
FDR at 3% rate:  55.59
OOT
FDR at 3% rate:  53.44


In [46]:
RunForest(150, 10, 200, 'entorpy', X_train, y_train)

Train
FDR at 3% rate:  55.51
Test
FDR at 3% rate:  54.76
OOT
FDR at 3% rate:  53.35


In [47]:
RunForest(150, 25, 300, 'entorpy', X_train, y_train)

Train
FDR at 3% rate:  55.73
Test
FDR at 3% rate:  55.06
OOT
FDR at 3% rate:  53.48


In [48]:
RunForest(150, 30, 400, 'entorpy', X_train, y_train)

Train
FDR at 3% rate:  55.67
Test
FDR at 3% rate:  55.12
OOT
FDR at 3% rate:  53.52


In [49]:
RunForest(100, 20, 400, 'gini', X_train, y_train)

Train
FDR at 3% rate:  55.71
Test
FDR at 3% rate:  55.12
OOT
FDR at 3% rate:  53.52


In [50]:
RunForest(50, 25, 300, 'gini', X_train, y_train)

Train
FDR at 3% rate:  55.65
Test
FDR at 3% rate:  55.06
OOT
FDR at 3% rate:  53.44


In [51]:
RunForest(50, 25, 300, 'gini', X_train, y_train)

Train
FDR at 3% rate:  55.69
Test
FDR at 3% rate:  54.99
OOT
FDR at 3% rate:  53.56


# Boosted Trees

In [ ]:
# XGBoost

In [ ]:
xgbs = XGBClassifier(n_jobs =  8, eta = 0.05, 
                     max_depth = 5, 
                     n_estimators = 600, subsample = 1,
                     eval_metric = 'auc')
model = xgbs.fit(X_train_under50, y_train_under50)

In [ ]:
y_pred = model.predict_proba(X_train)
print('Train')
findFDR(y_train, y_pred[:,1])

In [ ]:
y_pred = model.predict_proba(X_test)
print('Test')
findFDR(y_test, y_pred[:,1])

In [ ]:
y_pred = model.predict_proba(ootX)
print('OOT')
findFDR(ootY, y_pred[:,1])